# Self-adaptive Evolution Strategy (SAES)

$\newcommand{\R}{\mathbb{R}}$
$\newcommand{\E}{\mathbb{E}}$

$\newcommand{\vs}[1]{\boldsymbol{#1}}$
$\newcommand{\ms}[1]{\boldsymbol{#1}}$

## The SAES algorithm

In the following algorithm ${\cal{N}}$ denotes some independent standard Gaussian random variable, with dimension as required in equations above.

---

**Parameters:**
$K>0$,
$\zeta\geq 0$,
$\lambda>\mu>0$
and a dimension $d>0$.

**Input:**
an initial parent population $x_{1,i} \in \R^d$
and an initial $\sigma_{1,i} = 1$ with $i \in \{1, \dots, \mu \}$.

---

$n\leftarrow 1$

**while** (stop condition) **do**

$\quad$ Generate $\lambda$ individuals $i_j$ independently with $j \in \{ 1, \dots, \lambda \}$ using
    
\begin{eqnarray}
    \sigma_j & = & \sigma_{n, mod(j-1, \mu) + 1} \times \exp\left( \frac{1}{2d} \cal{N} \right) \\
    i_j      & = & \vs{x}_{n, mod(j-1, \mu) + 1} + \sigma_j \cal{N}.
\end{eqnarray}
    
$\quad$ Evaluate each of them $\lceil Kn^\zeta \rceil$ times and average their fitness values

$\quad$ Define $j_1, \dots, j_{\lambda}$ so that

$$
\E_{\lceil Kn^\zeta \rceil}[f(i_{j_1})]\leq \E_{\lceil Kn^\zeta \rceil}[f(i_{j_2})] \leq \dots \leq \E_{\lceil Kn^\zeta \rceil}[f(i_{j_{\lambda}})]
$$

$\quad$ where $\E_m$ denotes a sample average over $m$ resamplings.

$\quad$ Update: compute $\vs{x}_{n+1, k}$ and $\sigma_{n+1, k}$ using

\begin{eqnarray}
\sigma_{n+1,k}   &=& \sigma_{j_{k}}, \quad k \in \{1, \dots, \mu\}\\
{\vs{x}_{n+1,k}} &=& i_{j_{k}},      \quad k \in \{1, \dots, \mu\}
\end{eqnarray}

$\quad$ $n\leftarrow n+1$

**end while**

For more information, see http://www.scholarpedia.org/article/Evolution_strategies.

## A Python inplementation

The following implementation has been taken from there: https://github.com/jeremiedecock/pyai/blob/a61a866ead261e470785cc02c9d5aa6deeac80ee/pyai/optimization/standalone_implementations/saes/saes.py.

In [ ]:
#!/usr/bin/env python

"""
This is a simple Python implementation of the (mu/mu_I, lambda)-sigmaSA-ES
as discussed in 
http://www.scholarpedia.org/article/Evolution_Strategies
Based on
https://homepages.fhv.at/hgb/downloads/mu_mu_I_lambda-ES.oct
"""

import math
import numpy as np
import random

MU = 3                    # number of parents
LAMBDA = 12               # number of offspring

X_INIT = np.ones(2)       # initial parent vector
SIGMA_INIT = 1            # initial global mutation strength sigma

SIGMA_MIN = 1e-10         # ES stops when sigma is smaller than sigma_min

###########################################################

# Function to be optimized (sphere test function as an example)
# individual: the individual to evaluate

def fitness(individual):
    return sum(individual.x**2)

###########################################################

# The individual class
# x: the individual's value
# sigma: the individual's sigma
class Individual():
    def __init__(self, x, sigma):
        self.x = x
        self.sigma = sigma
        self.cost = fitness(self)

    def __str__(self):
        return "{0} {1} {2}".format(self.x, self.sigma, self.cost)

###########################################################

# This sorts the population according to the individuals' fitnesses
# pop: a list of Individual objects
def select_individuals(pop):
    pop.sort(key=lambda indiv: indiv.cost, reverse=False)
    return pop[:MU]

# This performs intermediate (multi-) recombination
# parents: a list of Individual objects
def recombine_individuals(parents):
    parents_y = np.array([indiv.x for indiv in parents])
    parents_sigma = np.array([indiv.sigma for indiv in parents])
    recombinant = Individual(parents_y.mean(axis=0), parents_sigma.mean())
    return recombinant

###########################################################

# Initialization
n = X_INIT.shape[0]         # determine search space dimensionality n   
tau = 1. / math.sqrt(2.*n)  # self-adaptation learning rate

# Initializing individual population
parent_pop = [Individual(X_INIT, SIGMA_INIT) for i in range(MU)]

# Evolution loop of the (mu/mu_I, lambda)-sigma-SA-ES
while parent_pop[0].sigma > SIGMA_MIN:
    offspring_pop = []
    recombinant = recombine_individuals(parent_pop) # TODO: BUG ? this statement may be in the next line
    for i in range(1, LAMBDA):
        offspring_sigma = recombinant.sigma * math.exp(tau * random.normalvariate(0,1))
        offspring_y = recombinant.x + offspring_sigma * np.random.normal(size=n)
        offspring = Individual(offspring_y, offspring_sigma)
        offspring_pop.append(offspring)
    parent_pop = select_individuals(offspring_pop)
    print(parent_pop[0])

# Remark: Final approximation of the optimizer is in "parent_pop[0].x"
#         corresponding fitness is in "parent_pop[0].cost" and the final 
#         mutation strength is in "parent_pop[0].sigma"

## Other inplementations

### PyAI

#### Import required modules

In [ ]:
# Init matplotlib

%matplotlib inline

import matplotlib
matplotlib.rcParams['figure.figsize'] = (8, 8)

In [ ]:
# Setup PyAI
import sys
sys.path.insert(0, '/Users/jdecock/git/pub/jdhp/pyai')

In [ ]:
import numpy as np
import time

from pyai.optimize import SAES

In [ ]:
# Plot functions
from pyai.optimize.utils import plot_contour_2d_solution_space
from pyai.optimize.utils import plot_2d_solution_space

from pyai.optimize.utils import plot_fx_wt_iteration_number
from pyai.optimize.utils import plot_err_wt_iteration_number
from pyai.optimize.utils import plot_err_wt_execution_time
from pyai.optimize.utils import plot_err_wt_num_feval

#### Define the objective function

In [ ]:
## Objective function: Rosenbrock function (Scipy's implementation)
#func = scipy.optimize.rosen

In [ ]:
# Set the objective function
#from pyai.optimize.functions import sphere as func
from pyai.optimize.functions import sphere2d as func

In [ ]:
saes = SAES()
res = saes.minimize(func)

### Octave/Matlab

An external Octave (Matlab) implementation is available there: https://homepages.fhv.at/hgb/downloads/mu_mu_I_lambda-ES.oct.